In [ ]:
import os
import json

from openai import OpenAI


In [ ]:
with open("../../config/keys.json", "r") as file:
    os.environ['OPENAI_API_KEY'] = json.load(file)["openai_api_key"]

In [ ]:
def judge_output():
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

    response = client.responses.create(
        model="gpt-4o",
        instructions="You are a coding assistant that talks like a pirate.",
        input="How do I check if a Python object is an instance of a class?",
    )

    print(response.output_text)